# How to Test Relational Fusion Network and Basline Algorithms

In [1]:
%load_ext autoreload
%autoreload 2

### Test
First, define test city sets.

In [2]:
# train_cities = ["busan", "daejeon", "goyang", "gwangju", "incheon", "changwon", "seongnam", "cheongju"]
test_cities = ["daegu", "suwon", "ulsan", "yongin"]

Then define models to test. We used 4 RFN vaiations and 4 baseline models(GAT, GCB, GraphSAGE and MLP).

In [3]:
from custom.rfn_trainer import RFNTrainer
from custom.gcn_trainer import GCNTrainer
from custom.mlp_trainer import MLPTrainer
from custom.utils import * 

# RFN models
rfn_trainer_ia = RFNTrainer(fusion='interactional', aggregator='attentional')
rfn_trainer_in = RFNTrainer(fusion='interactional', aggregator='non-attentional')
rfn_trainer_aa = RFNTrainer(fusion='additive', aggregator='attentional')
rfn_trainer_an = RFNTrainer(fusion='additive', aggregator='non-attentional')

# Baseline models
gcn_trainer_gat = GCNTrainer("gat")
gcn_trainer_gcn = GCNTrainer("gcn")
gcn_trainer_gsg = GCNTrainer("graphsage")
mlp_trainer = MLPTrainer()


rfn_models = [rfn_trainer_ia, rfn_trainer_in, rfn_trainer_aa, rfn_trainer_an]
baseline_models = [gcn_trainer_gat, gcn_trainer_gcn, gcn_trainer_gsg, mlp_trainer]
target_models = rfn_models + baseline_models

Using backend: pytorch


Add city information to trainer. Note that MXNet is used for RFN while dgl+PyTorch is used for the rest.

In [4]:
for city_name in test_cities:
    G = load_city_graph(city_name)
    rfn_city, dgl_city = generate_required_city_graph(city_name, G)
    for t in target_models:
        if isinstance(t, RFNTrainer):
            t.add_city(rfn_city, train=False)
        else:
            t.add_city(dgl_city, train=False)

[41134, 35295, 53753, 34446]
Primal V,E: (18339, 52910), Dual V,E: (52910, 164628)
[17056, 14445, 21915, 14132]
Primal V,E: (7162, 21254), Dual V,E: (21254, 67548)
[20713, 17365, 27460, 17380]
Primal V,E: (8822, 26070), Dual V,E: (26070, 82918)
[11159, 7614, 15077, 7594]
Primal V,E: (5659, 14447), Dual V,E: (14447, 41444)


Now build network and load data.

In [5]:
load_data_folder = 'model_data/20200619_182329'
for t in target_models:
    t.build()
    t.load_params("%s/%s" % (load_data_folder, t.name))

FeatureInfo(no_node_features=2, no_edge_features=16, no_between_edge_features=4)
FeatureInfo(no_node_features=2, no_edge_features=16, no_between_edge_features=4)
FeatureInfo(no_node_features=2, no_edge_features=16, no_between_edge_features=4)
FeatureInfo(no_node_features=2, no_edge_features=16, no_between_edge_features=4)


Test on each road networks.

In [9]:
test_results = {}
for t in target_models:
    test_results[t.name] = t.test(verbose = False)

Now print results.

In [10]:
a = pd.DataFrame.from_dict(test_results)
print(a.transpose())

                daegu     suwon     ulsan    yongin
rfn_int_att  0.013797  0.006467  0.017539  0.008208
rfn_int_non  0.017247  0.012515  0.019019  0.028123
rfn_add_att  0.015978  0.010970  0.019676  0.028279
rfn_add_non  0.015818  0.010134  0.019537  0.029834
gat          0.041885  0.027642  0.037875  0.052309
gcn          0.046810  0.035997  0.045919  0.065880
graphsage    0.044869  0.032357  0.031119  0.044752
mlp          0.025466  0.012247  0.022358  0.014048


## References
[1]: Tobias Skovgaard Jepsen, Christian S. Jensen, and Thomas Dyhre Nielsen.  "Graph Convolutional Networks for Road Networks." In _27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems (SIGSPATIAL ’19)_. 2019.

[2]: Tobias Skovgaard Jepsen, Christian S. Jensen, and Thomas Dyhre Nielsen. "Graph Convolutional Networks for Road Networks." _arXiv e-prints_, arXiv:1908.11567. 2019.

[3]: Hamilton, Will, Zhitao Ying, and Jure Leskovec. "Inductive representation learning on large graphs." _Advances in Neural Information Processing Systems_. 2017.